In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [6]:
df = pd.read_csv('Loss_plot/loss_plot_google-bert_bert-base-cased.csv')
df[['epoch', 'eval_loss', 'train_loss']]

,epoch,eval_loss,train_loss
0,1.0,NaN,NaN
1,1.0,2.854533,NaN
2,2.0,NaN,NaN
3,2.0,2.752703,NaN
4,3.0,NaN,NaN
...,...,...,...
56,29.0,NaN,NaN
57,29.0,1.984650,NaN
58,30.0,NaN,NaN
59,30.0,1.985869,NaN


In [12]:
df[['loss']].describe()

,loss
count,30.000000
mean,1.375527
std,0.854912
min,0.473100
25%,0.607000
50%,1.090300
75%,2.118050
max,2.913600
